In [1]:
import pmdarima as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sq
from sklearn.metrics import median_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
# Mean absolute percent error
def MAPE(Y_true,Y_Predicted):
    mape = np.mean(np.abs((Y_true - Y_Predicted)/Y_true))*100
    return mape
# Simple percent split --- !!! applies log to train_y
# would be good to add param for choosing split
def per_split(df):
    ave_crime = df['Number of crimes'].mean()
    train = df.iloc[:100]
    test = df.iloc[100:].fillna(ave_crime)
    train_x = train[['Number of crimes', 1]] #1: year, including year increases mape by 13.9966 - 11.7564 percent
    train_y = train['next_num_crimes'].apply(lambda cell: ave_crime if cell == np.nan else np.log10(cell))
    test_x = test[['Number of crimes', 1]]
    test_y = test['next_num_crimes']
    return train_x, train_y, test_x, test_y
def per_split_in(df, split):
    ave_crime = df['Number of crimes'].mean()
    train = df.iloc[:split]
    test = df.iloc[split:].fillna(ave_crime)
    train_x = train[['Number of crimes', 1, 0]] #1: year, including year increases mape by 13.9966 - 11.7564 percent
    train_y = train['next_num_crimes'].apply(lambda cell: ave_crime if cell == np.nan else np.log10(cell))
    test_x = test[['Number of crimes', 1, 0]]
    test_y = test['next_num_crimes']
    return train_x, train_y, test_x, test_y
def eval(test_y, pred):
    mae = mean_absolute_error(test_y, pred)
    mape = MAPE(test_y, pred)
    rsme = sqrt(mean_squared_error(test_y, pred))
    print(f'MAE: {mae}, MAPE: {mape},RSME: {rsme}')
# log = lambda cell: ave_crime if cell == np.nan else np.log10(cell)
undo_log = np.vectorize(lambda x: 10 ** x)

df_police_force = pd.read_csv('police force dataset grouped and preprocessed.csv', sep=';')

def police_force_to_time_series(df):
    # Input police force
    x = input('Which police force?')

    filtered = df[df['Falls within'] == x]
    df = filtered.groupby('County')['COUNT(*)'].sum()
    df = df.to_frame()
    df.reset_index(inplace=True)
    max_value = df['COUNT(*)'].max()
    list_counties = []
    # Adding all the counties that have equal to or greater than 10% of the crimes of the max county.
    for i in range(len(df)):
        if df.loc[i, 'COUNT(*)'] > (max_value / 10):
            # Prevalent county if more than 10 percent of county with most crimes in the county.
            list_counties.append(df.loc[i, 'County'])

    # Append the other category to the list_counties
    list_counties.append('Other')

    list_df_ts = []
    # Making time series dataframes for each county
    for county in list_counties:
        county_df = filtered[filtered['County'] == county]
        count_per_month_for_county = county_df.groupby('Month')['COUNT(*)'].sum()
        count_per_month_for_county = count_per_month_for_county.to_frame()
        count_per_month_for_county.reset_index(inplace=True)
        count_per_month_for_county = count_per_month_for_county.rename(columns={'COUNT(*)': 'Number of crimes'})
        # Change to datetime, in order to make it a valid time series
        count_per_month_for_county['Month'] = pd.to_datetime(count_per_month_for_county['Month'])
        list_df_ts.append((county, count_per_month_for_county))

    return list_df_ts
from sklearn import metrics

def r2_adj_from_r2(r2,n,k = 3):
    #n is the number of values (samples) in the data
    #k is the number of variables in the data --> normally 1, namely the time
    a1= (1-r2) * (1-n)
    a2= n-k-1
    a3= 1
    r2_adj = a3 - (a1/a2)
    return r2_adj

# Performance measures based on y_true and y_pred
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print(f'Number of months in test data: {len(y_true)}')
    print('Evaluation metric results:-')
    print(f'(Mean Squared Error) MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'(Mean Absolute Error) MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'(Root Mean Square Error) RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'(Mean Absolute Percentage Error) MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    # Computing R^2 by hand:
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_true = y_true.mean()
    R2 = 1 - (sum((y_true - y_pred)**2) / sum((y_true - mean_true)**2))
    # Same result with
    print(f'(R-Squared) R2 is : {R2}')
    adj_R2 = r2_adj_from_r2(R2,len(y_pred))
    print(f'(adjusted R-Squared) adj-R2 is : {adj_R2}',end='\n\n')


In [2]:

list_ts = police_force_to_time_series(df_police_force) # Avon and Somerset Constabulary

ts_county = list_ts[1][1]
ts_county['Number of crimes'] = ts_county['Number of crimes'].astype(float)
ts_month_as_index = ts_county.set_index('Month')
months = [x.month for x in ts_month_as_index.index]
years = [x.year for x in ts_month_as_index.index]
X = pd.DataFrame(np.array([months, years]).T)
ts_county['next_num_crimes'] = ts_county['Number of crimes'].shift(-1)

ave_crime = ts_county['Number of crimes'].mean()
log = lambda cell: ave_crime if cell == np.nan else np.log10(cell)

ts_county['next_num_crimes'].fillna(ts_county['next_num_crimes'].mean(), inplace = True)
t = X.join(ts_county)   # include year in prediction (year and month but month makes it worse)


Which police force?Avon and Somerset Constabulary


In [3]:
def chain_ar_logdif(data):
    """input: df w/ 'Month', 'Number of crimes', 'next_num_crimes'
    return: ar model for log dif
        """
    #data['log_dif'] = data['next_num_crimes'].apply(log) - data['Number of crimes'].apply(log)
    data['log'] = data['Number of crimes'].apply(log)
    data['next_num_crimes'].fillna(data['next_num_crimes'].mean(), inplace = True)
    #data['log_dif'].fillna(data['log_dif'].mean(), inplace = True)

    ts_split = TimeSeriesSplit(n_splits=5)
    #ts_mean = data['log_dif'][:26].mean()
    ts_mean = data['log'][:26].mean()
    forecasts = np.full(26, ts_mean)  # first 25 "predictions" = mean # of crimes -- makes graph nicer / fix indexing
    i = 0
    for train_index, test_index in ts_split.split(data['Number of crimes']):
        print("TRAIN:", train_index, "TEST:", test_index)
        train_end = train_index[-1]
        if i ==0:
            model = pm.auto_arima(data['log'].iloc[:train_end], x='Month', seasonal=True, m=12)
            i = 1
            print('FIRST MODEL')
        else:

            model.update(data['log'].iloc[:train_end], x='Month')
            i += 1
            print(f'MODEL: {i}')
        forecast = model.predict(len(test_index))
        forecasts = np.append(forecasts, forecast)
        print(forecast, forecasts)
        timeseries_evaluation_metrics_func(undo_log(data['log'].iloc[train_end:test_index[-1]]), undo_log(forecast))

    plt.plot(pd.Series(forecasts), c='blue')
    plt.plot(data['log'], c='red')
    plt.show()
    # print(mean_absolute_error(data['log_dif'][26:], forecasts[26:]))   # attempt to eval model
    return model
ar_log_dif = chain_ar_logdif(t)
undo_log(ar_log_dif.predict(5))
x = 111
test_one = ar_log_dif.update(t['log'].iloc[:x], x='Month')
timeseries_evaluation_metrics_func(t['Number of crimes'].iloc[x:], undo_log(test_one.predict(18)))


TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25] TEST: [26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]


ValueError: zero-size array to reduction operation maximum which has no identity

In [4]:
def rf_reg(data, train_size):
    """
    :param data:
    requires columns: 1, num crimes, next num
    :return:
    rf regressor model
    """
    train_x, train_y, test_x, test_y = per_split_in(data, train_size) # 120 -> mape: 4.8 increase training means less test data -> super accurate
    xgb_model = xgb.XGBRFRegressor().fit(train_x, train_y,
                                         eval_set=[(train_x, train_y), (test_x, test_y)])
    # evals_result = xgb_model.evals_result()
    # print(evals_result)
    # pred = xgb_model.predict(test_x)
    # eval(test_y, undo_log(xgb_model.predict(test_x)))
    return xgb_model, test_x, test_y
rf_model, test_x, test_y = rf_reg(t, 90)
pred = undo_log(rf_model.predict(test_x))
eval(test_y, pred)

[0]	validation_0-rmse:0.01214	validation_1-rmse:5074.93506
MAE: 288.498568509639, MAPE: 6.052369407703416,RSME: 387.6981405579882


In [5]:

df_police_force = pd.read_csv('police force dataset grouped and preprocessed.csv', sep=';')
df_police_force_p_month = df_police_force.groupby('Month')['COUNT(*)'].sum()
df_police_force_p_month = df_police_force_p_month.to_frame()
df_police_force_p_month.reset_index(inplace = True)
df_police_force_p_month = df_police_force_p_month.rename(columns = {'COUNT(*)': 'Number of crimes'})
# Change to datetime, in order to make it a valid time series
df_police_force_p_month['Month'] = pd.to_datetime(df_police_force_p_month['Month'], format="%Y-%m", exact=True)

df_police_force_p_month['next_num_crimes'] = df_police_force_p_month['Number of crimes'].shift(-1)

month_index = df_police_force_p_month.set_index('Month')
months = [x.month for x in month_index.index]
years = [x.year for x in month_index.index]
X = pd.DataFrame(np.array([months, years]).T)